In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# **Question 1 :**

# **`Accessing data from wikipedia`**

In [0]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",header=0)


# Storing Data in Dataframe

In [5]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
toronto_data = pd.DataFrame(columns=column_names)
toronto_data  = data[0]
toronto_data = toronto_data.rename(columns={"Postal code": "PostalCode"})
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [6]:
toronto_data.shape

(180, 3)

# Dropping all rows with Borough = "Not Assigned"

In [0]:
x = toronto_data.index[toronto_data['Borough']=="Not assigned"].tolist()  # List of index having borough = not assigned

In [8]:
toronto_data = toronto_data.drop(toronto_data.index[x]) #dropping all the rows with this borough = not assigned.
toronto_data = toronto_data.reset_index(drop=True)   # Resetting the index
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [9]:
toronto_data.shape

(103, 3)

# Separating multiple neighorhoods by " , "

In [0]:
x = []
for i in toronto_data.Neighborhood:
  x.append(i.replace(" /",","))

df = pd.DataFrame({"Neighborhood":x})
toronto_data.update(df)

# Question 1 Final Ouput

In [11]:
toronto_data.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [12]:
toronto_data.shape

(103, 3)

# **Question 2 :**

# Reading the Latitude, Longitude file Provided.

In [24]:
df = pd.read_csv("http://cocl.us/Geospatial_data")
df.head()

ERROR! Session/line number was not unique in database. History logging moved to new session 59


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Finding Latitude, Longitude for Postal Codes using provided file.

In [0]:
# Creating an Empty list
column_names = ['PostalCode', 'Latitude', 'Longitude'] 

# Creating an Empty Dataframe
l1 = pd.DataFrame(columns=column_names)

# Matching the Postal codes from toronto_data and provided file to extract exact Latitude, Longitude for specific postal code.
for i in toronto_data.index:
  for j in df.index:
    if toronto_data.PostalCode[i]==df["Postal Code"][j]:
      l1 = l1.append({"PostalCode":toronto_data.PostalCode[i],   # Appending all info to empty dataframe.
                "Latitude":df.Latitude[j],
                "Longitude":df.Longitude[j]},ignore_index=True)


In [60]:
l1.head(10)


,PostalCode,Latitude,Longitude
0,M3A,43.753259,-79.329656
1,M4A,43.725882,-79.315572
2,M5A,43.654260,-79.360636
3,M6A,43.718518,-79.464763
4,M7A,43.662301,-79.389494
5,M9A,43.667856,-79.532242
6,M1B,43.806686,-79.194353
7,M3B,43.745906,-79.352188
8,M4B,43.706397,-79.309937
9,M5B,43.657162,-79.378937


# Adding Latitude, Longitude to toronto_data

In [0]:
toronto_data["Latitude"] = l1["Latitude"]      # Adding Latitude, Longitude to toronto_data.
toronto_data["Longitude"] = l1["Longitude"]

# Question 2 Final Output

In [61]:
toronto_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
